# "DETECTRON2 USING SEGMENTATION"

> "DETECTRON2 USING SEGMENTATION"

- toc:true
- branch: master
- badges: true
- comments: true
- author: HyunsooKim
- categories: [jupyter, python]

## DETECTRON2를 이용한 SEGMENTATION

사전 작업 

segmentation를 하기 위해 labelme를 이용하여 이미지 파일의 물 객체를 폴리곤 형태로 지정한다.

detection를 하기 위해 LabelImg를 이용해서 물의 객체를 지정한다.


사용자의 구글 드라이브의 파일을 불러오기 위해서 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Detectron2를 colab에 설치

In [ ]:
!pip install pyyaml==5.1
!pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
# pytorch를 최신 버전인 1.9가 아닌 1.8를 사용하는 이유는 아직 detectron2에 pytorch 1.9 패키지가 출시되지 않았기 떄문이다

In [ ]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html

In [ ]:
# 설치된 패키지들을 import함수를 이용해서 불러오는 작업
import torch
assert torch.__version__.startswith("1.8") 
import torchvision
import cv2

Detectron2에 필요한 데이터를 등록

In [ ]:
# config하기 위해 필요한 패키지 불러오기 
import os
import numpy as np
import json
import random
import matplotlib.pyplot as plt
%matplotlib inline 

from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog

In [ ]:
# 라벨링한 json파일의 class 및 기본 정보를 얻는 def 함수  
def get_data_dicts(directory, classes):
    dataset_dicts = []
    for filename in [file for file in os.listdir(directory) if file.endswith('.json')]:
        json_file = os.path.join(directory, filename)
        with open(json_file) as f:
            img_anns = json.load(f)

        record = {}
        
        filename = os.path.join(directory, img_anns["imagePath"])
        
        record["file_name"] = filename
        #record["height"] = 512
        #record["width"] = 384
      
        annos = img_anns["shapes"]
        objs = []
        for anno in annos:
            px = [a[0] for a in anno['points']] # x coord
            py = [a[1] for a in anno['points']] # y coord
            poly = [(x, y) for x, y in zip(px, py)] # segmentation을 위한 poly
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": classes.index(anno['label']),
                "iscrowd": 0
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [ ]:
for d in ["train", "test"]:
    DatasetCatalog.register("images_" + d, lambda d=d: get_water_dicts("for seg/" + d)) 
    MetadataCatalog.get("bimages_" + d).set(thing_classes=["water"])
water_metadata = MetadataCatalog.get("water_train")

In [ ]:
classes = ['water']

data_path = '/content/drive/MyDrive/project_datacampus/dataset/for_seg/'

for d in ["train", "test"]:
    DatasetCatalog.register(
        "category_" + d, 
        lambda d=d: get_data_dicts(data_path+d, classes)
    )
    MetadataCatalog.get("category_" + d).set(thing_classes=classes)

microcontroller_metadata = MetadataCatalog.get("category_train")

Detectron2의 Instance Segmentation 모델을 학습시키기

In [ ]:
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode, Visualizer

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("category_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2 
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 2000
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)

In [ ]:
trainer.train()

train를 통해 나온 모델을 사용한 추론(Inference)

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "/content/drive/MyDrive/project_datacampus/dataset/for_seg/model_final_last.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.75 
cfg.DATASETS.TEST = ("skin_test", )
predictor = DefaultPredictor(cfg)

In [ ]:
test_dataset_dicts = get_data_dicts(data_path+'test', classes)

In [ ]:
for d in random.sample(test_dataset_dicts, 3):    
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)
    v = Visualizer(img[:, :, ::-1],
                   metadata=microcontroller_metadata, 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW 
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize = (14, 10))
    plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
from detectron2.utils.visualizer import ColorMode
from google.colab.patches import cv2_imshow
dataset_dicts = get_data_dicts("/content/drive/MyDrive/project_datacampus/dataset/for_seg/test",'water') 
for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  
    v = Visualizer(im[:, :, ::-1],
                   metadata=water_metadata, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW   
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

tensorboard를 통해 학습된 모델의 세부 성능 지표를 확인

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output

이미지 파일을 이용한 면적 구하는 과정 

In [ ]:
# colab에 detectron2 폴더 생성
!git clone https://github.com/facebookresearch/detectron2
%cd /content/detectron2

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os
from google.colab.patches import cv2_imshow

cfg = get_cfg()
cfg.merge_from_file("/content/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("water",)
cfg.DATASETS.TEST = ()   
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/project_datacampus/dataset/for_seg/model_final_last.pth" 
cfg.SOLVER.BASE_LR = 0.025
cfg.SOLVER.MAX_ITER = 2000   
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # 1 classes (water)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [ ]:
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/project_datacampus/dataset/for_seg/model_final_last.pth" 
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.75  # 모델의 정확도 설정 파트 75%로 설정됨 
cfg.DATASETS.TEST = ("water", )
predictor = DefaultPredictor(cfg)

In [ ]:
path = "/content/drive/MyDrive/project_datacampus/dataset/for_seg/test/image_200.jpg"
im = cv2.imread(path)
outputs = predictor(im)

In [ ]:
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
mask_array = outputs["instances"].pred_masks.cpu().numpy()
num_instances = mask_array.shape[0]
mask_array = np.moveaxis(mask_array, 0, -1)

mask_array_instance = []
output = np.zeros_like(im) #black

#print('output',output)
for i in range(num_instances):
    mask_array_instance.append(mask_array[:, :, i:(i+1)])
    # print(mask_array_instance)
    output = np.where(mask_array_instance[i] == True, 255, output)

cv2_imshow(output)

In [ ]:
### 영상 읽기 
img_raw = output.copy()
### color 영상을 Grayscale 영상으로 변환
img_gray = cv2.cvtColor(img_raw,cv2.COLOR_BGR2GRAY)
### Otsu's thresholding
_, img_binary = cv2.threshold(img_gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU) 
### 닫기 연산 (잡음제거)
kernel = np.ones((3,3),np.uint8)
img_morph = cv2.morphologyEx(img_binary,cv2.MORPH_CLOSE,kernel)
### 컨투어 찾기 - 꼭짓점 좌표 cv2.findContours(src,mode,method,contours,hierarchy,offset)
contours,hierarchy = cv2.findContours(img_morph,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
### 컨투어 그리기 -cv2.drawContours(img,contours,contourldx,color,thickness) contourldx = -1 모든 컨투어 :표시
img_out = img_raw.copy()
_=cv2.drawContours(img_out,contours,-1,(0,0,200),10) #-1로 고려 필요할듯 전체를 컨투어 해야하기에 

region = 0
for cnt in contours:
    area = cv2.contourArea(cnt)
    region = region + area
AREA=round(region/(img_out.shape[0]*img_out.shape[1])*100,2)
# AREA의 면적 퍼센트에 따라 글씨의 색상을 바꾸어서 경보단계를 표시 
if AREA <= 10:
  color = (255,0,0)
elif AREA <= 20:
  color = (0,255,255)
else:
  color = (0,0,255)
font = cv2.FONT_HERSHEY_SIMPLEX
#color = (255, 0, 0)
fontScale = img_out.shape[0]/200
cv2.putText(img_out, f"{AREA:.2f}%", (20,50), font, fontScale, color,10) #사진의 크기마다 조절 혹은 전체 resize 
### 영상 출력 
plt.imshow(cv2.cvtColor(img_out,cv2.COLOR_BGR2RGB))
plt.axis('on') #on하면 눈금값도 나옴 
plt.show

contour를 통해 segmentation면적 구하는 공식 

In [ ]:
region = 0
for cnt in contours:
    area = cv2.contourArea(cnt)
    region = region + area
display(region)
display(round(region/(img_out.shape[0]*img_out.shape[1])*100,2)) #반올림 처리 

영상 디텍팅


In [ ]:
# 유튜브의 영상 확인
from IPython.display import YouTubeVideo, display
video = YouTubeVideo("EixdqWFs-Os", width=500) # 5R3vOr0iyZI(도로침수) //mv5LtkXBdso(도로) //AsXMIBHxPLY(댐)
display(video)

In [ ]:
# 유튜브에서 영상 다운로드
!pip install youtube-dl
!pip uninstall -y opencv-python opencv-contrib-python
!apt install python3-opencv
!youtube-dl https://www.youtube.com/watch?v=EixdqWFs-Os -f 22 -o video.mp4
!ffmpeg -i video.mp4 -t 00:00:8 -c:v copy video-clip.mp4

In [ ]:
# 다운로드한 영상을 segmentation 진행 
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.utils.visualizer import ColorMode, Visualizer
import tqdm

# 비디오의 속성 추출
video = cv2.VideoCapture('video-clip.mp4') #유튜브가 아닌 다른 소스로 저장된 mp4도 넣으면 가능 
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
frames_per_second = video.get(cv2.CAP_PROP_FPS)
num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))


video_writer = cv2.VideoWriter('out.mp4', fourcc=cv2.VideoWriter_fourcc(*"mp4v"), fps=float(frames_per_second), frameSize=(width, height), isColor=True)

v = VideoVisualizer(MetadataCatalog.get(cfg.DATASETS.TEST[0]), ColorMode.IMAGE)

def runOnVideo(video, maxFrames):
    """ Runs the predictor on every frame in the video (unless maxFrames is given),
    and returns the frame with the predictions drawn.
    """

    readFrames = 0
    while True:
        hasFrame, frame = video.read()
        if not hasFrame:
            break

        # 프레임의 예측값 얻기
        outputs = predictor(frame)

        # 프레임의 컬러가 있도록 
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        # instance segmentation를 통해 예측 결과를 표시
        visualization = v.draw_instance_predictions(frame, outputs["instances"].to("cpu"))

        # Matplotlib RGB 포맷을 OpenCV BGR 포맷으로 변경
        visualization = cv2.cvtColor(visualization.get_image(), cv2.COLOR_RGB2BGR)

        yield visualization

        readFrames += 1
        if readFrames > maxFrames:
            break

for visualization in tqdm.tqdm(runOnVideo(video, num_frames), total=num_frames):

    video_writer.write(visualization)

video.release()
video_writer.release()
cv2.destroyAllWindows()

In [ ]:
# segmentation하여 나온 결과 영상 저장 
from google.colab import files
files.download('/content/detectron2/out.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

저장된 비디오 캡쳐(10 or 30초)

In [ ]:
#1초당 프레임 체크 
filepath = '/content/drive/MyDrive/project_datacampus/dataset/for_seg/video/road_origin.mp4'
video = cv2.VideoCapture(filepath)
fps = video.get(cv2.CAP_PROP_FPS)
print(fps)
frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)
display(frame_count)

비디오를 일정 프레임마다 캡쳐하여 이미지로 변경 

In [ ]:
import cv2

# 영상의 의미지를 연속적으로 캡쳐할 수 있게 하는 class
# 영상이 있는 경로
vidcap = cv2.VideoCapture('/content/drive/MyDrive/project_datacampus/dataset/for_seg/video/road_origin.mp4') ####start from here

count = 0

while(vidcap.isOpened()):
    ret, image = vidcap.read()
    # 이미지 사이즈 960x540으로 변경
    #image = cv2.resize(image, (960, 540)) #512,384
    #image = cv2.rotate(image, cv2.ROTATE_180)
     
    saveme = open('/content/drive/MyDrive/project_datacampus/dataset/for_seg/road/cap/frame.txt', 'a') 
    # 일정시간마다 하나씩 이미지 캡쳐
    if(int(vidcap.get(1)) % 29 == 0): 
        print('Saved frame number : ' + str(int(vidcap.get(1))))
        print(str(int(vidcap.get(1))),file=saveme)
        # 추출된 이미지가 저장되는 경로
        cv2.imwrite("/content/drive/MyDrive/project_datacampus/dataset/for_seg/road/cap/frame%d.jpg" % count, image)
        #print('Saved frame%d.jpg' % count)
        count += 1
        if (vidcap.get(1)) == 1044:
          break
        
vidcap.release() 

캡쳐한 이미지를 segmentation한 이후 contour를 통해 면적 비율 구하기 

In [ ]:
from detectron2.config import get_cfg
import os
from google.colab.patches import cv2_imshow
from PIL import Image, ImageDraw, ImageFont
import glob

#cfg = get_cfg()
#cfg.merge_from_file("/content/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
#cfg.DATASETS.TRAIN = ("water",)
#cfg.DATASETS.TEST = ()   # no metrics implemented for this dataset
#cfg.DATALOADER.NUM_WORKERS = 2
#cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/project_datacampus/dataset/for_seg/model_final_last.pth"  # initialize from model zoo 경로확인!!
#cfg.SOLVER.IMS_PER_BATCH = 2
#cfg.SOLVER.BASE_LR = 0.025
#cfg.SOLVER.MAX_ITER = 2000   # 300 iterations seems good enough, but you can certainly train longer
#cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset
#cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # 1 classes (person)

#os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

#cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/K-Water/for seg/model_final_last.pth" # 여기부분은 본인의 model이저장된 경로로 수정해줍니다.
#cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.75  # set the testing threshold for this model
#cfg.DATASETS.TEST = ("water", )
#predictor = DefaultPredictor(cfg)

file_path = '/content/drive/MyDrive/project_datacampus/dataset/for_seg/road/cap/'
list_images = glob.glob(file_path + '*.jpg')
j=0
k=0
for image in list_images:
  im = Image.open(image)
  im = np.array(im)
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
  v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2.imwrite('/content/drive/MyDrive/project_datacampus/dataset/for_seg/road/seg/frame%d.jpg'%j,v.get_image())
  j=j+1
  mask_array = outputs["instances"].pred_masks.cpu().numpy()
  num_instances = mask_array.shape[0]
  mask_array = np.moveaxis(mask_array, 0, -1)

  mask_array_instance = []
  output = np.zeros_like(im) #black
  for i in range(num_instances):
    mask_array_instance.append(mask_array[:, :, i:(i+1)])
    output = np.where(mask_array_instance[i] == True, 255, output)
  img_raw = output.copy()
  ### color 영상을 Grayscale 영상으로 변환
  img_gray = cv2.cvtColor(img_raw,cv2.COLOR_BGR2GRAY)
  ### Otsu's thresholding
  _, img_binary = cv2.threshold(img_gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU) 
  ### 닫기 연산 (잡음제거)
  kernel = np.ones((3,3),np.uint8)
  img_morph = cv2.morphologyEx(img_binary,cv2.MORPH_CLOSE,kernel)
  ### 컨투어 찾기 - 꼭짓점 좌표 cv2.findContours(src,mode,method,contours,hierarchy,offset)
  contours,hierarchy = cv2.findContours(img_morph,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
  ### 컨투어 그리기 -cv2.drawContours(img,contours,contourldx,color,thickness) contourldx = -1 모든 컨투어 :표시
  img_out = img_raw.copy()
  _=cv2.drawContours(img_out,contours,-1,(0,0,200),10) 

  region = 0
  for cnt in contours:
    saveme = open('/content/drive/MyDrive/project_datacampus/dataset/for_seg/road/cap/output.txt', 'a') 
    area = cv2.contourArea(cnt)
    region = region + area
  AREA=round(region/(img_out.shape[0]*img_out.shape[1])*100,2)
  # AREA의 면적 퍼센트에 따라 글씨의 색상을 바꾸어서 경보단계를 표시 
  if AREA <= 30:
    color = (255,0,0)
  elif AREA <= 40:
    color = (0,255,255)
  else:
    color = (0,0,255)
  font = cv2.FONT_HERSHEY_SIMPLEX
  #color = (255, 0, 0)
  fontScale = img_out.shape[0]/200
  cv2.putText(img_out, f"{AREA:.2f}%", (20,150), font, fontScale, color,13) 
  display(AREA)
  print(AREA, file=saveme)

  #cv2_imshow(img_out[:, :, ::-1])
  cv2.imwrite("/content/drive/MyDrive/project_datacampus/dataset/for_seg/road/cnt/frame%d.jpg"%k,img_out)
  k=k+1

txt파일 두개 결합이후 plotting

In [ ]:
import pandas as pd
# txt형태의 파일을 csv형태의 파일로 로드
df1 = pd.read_table('/content/drive/MyDrive/project_datacampus/dataset/for_seg/road/cap/frame.txt',sep='\n',header=None,names=['fps'])
df2 = pd.read_table('/content/drive/MyDrive/project_datacampus/dataset/for_seg/road/cap/output.txt',sep='\n',header=None,names=['area'])
# 두개의 파일을 하나로 합쳐주기
df_result = pd.concat([df1,df2],axis=1)
#df_result = df_result[:89]
df_result

import matplotlib.pyplot as plt
import seaborn as sns

# seaborn를 이용한 plotting
fig, ax = plt.subplots(figsize=(13, 7)) 
sns.set_theme(style="darkgrid")
plt.title('Segmentation Area Ratio', fontsize=17)
sns.lineplot(df_result.fps, df_result.area)
ax.set_xlabel('fps', size=13)
ax.set_ylabel('AREA (%)', size=13)
plt.axhline(y=30, color='yellow', linewidth=1)
plt.axhline(y=40, color='orange', linewidth=1)
plt.axhline(y=50, color='red', linewidth=1)
ax.legend(['Ratio','alert 1','alert 2','alert 3'])
plt.show() 

In [ ]:
# 강의 면적 수치에 대한 정보 
display(df2.min())
display(df2.max())
display(df2.mean())

참고 출처
- <https://www.kaggle.com/code/ehabibrahim758/flood-segmentation-and-classification/notebook> 

- <https://github.com/facebookresearch/detectron2>

- <https://www.geumriver.go.kr/html/sumun/river_movie_all.jsp>

- <https://gilberttanner.com/blog/detectron2-train-a-instance-segmentation-model/>

- <https://www.mdpi.com/2076-3417/11/20/9691/htm>